In [1]:
from azure.identity import AzureCliCredential
from azure.ai.ml import MLClient
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes

# Use Azure CLI credential (since you're logged in via `az login`)
credential = AzureCliCredential()

ml_client = MLClient(
    credential=credential,
    subscription_id="6bd1f99e-e7cb-4226-b9d5-09433d793bda",
    resource_group_name="shafi1",
    workspace_name="shafi1"
)


In [2]:
# Step 1: Install H2O (uncomment if not already installed)
# !pip install h2o

# Step 2: Import required libraries
import h2o
from h2o.estimators.gbm import H2OGradientBoostingEstimator

# Step 3: Initialize H2O
h2o.init()

# Step 4: Load the Iris dataset (comes built-in with H2O)
iris = h2o.import_file("https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv")

# Step 5: Define target and features
target = "sepal_length"
features = iris.columns
features.remove(target)

# Step 6: Split data into train and test sets
train, test = iris.split_frame(ratios=[0.8], seed=1234)

# Step 7: Train the GBM regression model
 # Train Gradient Boosting Model
model = H2OGradientBoostingEstimator(ntrees=50, max_depth=5, learn_rate=0.1)
model.train(x=features, y=target, training_frame=train)


# Step 8: Make predictions on test set
predictions = model.predict(test)

# Step 9: Display sample predictions
print("Sample Predictions:")
print(predictions.head())

# Step 10: Evaluate model performance
performance = model.model_performance(test_data=test)
print("\nModel Performance on Test Set:")
print(f"R-squared: {performance.r2():.4f}")
print(f"MSE: {performance.mse():.4f}")
print(f"RMSE: {performance.rmse():.4f}")

# Step 11: Shut down H2O (optional)
# h2o.shutdown(prompt=False)


Checking whether there is an H2O instance running at http://localhost:54321. connected.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,2 days 0 hours 12 mins
H2O_cluster_timezone:,Asia/Kolkata
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.1
H2O_cluster_version_age:,"2 years, 4 months and 6 days"
H2O_cluster_name:,H2O_from_python_mdshafiuddin_qae39y
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.981 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
gbm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Sample Predictions:
  predict
  4.83635
  4.83635
  5.18013
  5.12249
  5.17126
  5.29605
  4.89153
  5.39518
  5.1054
  4.78155
[10 rows x 1 column]


Model Performance on Test Set:
R-squared: 0.7800
MSE: 0.1123
RMSE: 0.3350


In [22]:

from datetime import datetime
import os
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

output_directory = "./models"



model_filename = f"my_h2o_model_{timestamp}.zip"

full_mojo_path = os.path.join(output_directory, model_filename)

mojo_path = model.download_mojo(path=output_directory, get_genmodel_jar=True)

In [23]:
from datetime import datetime
import os

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Since you're running this script *inside* the 'model' folder,
# the output directory relative to the script is just '.' (current directory).
output_directory = "."

# No need to create the directory as you're already in it.

model_filename = f"my_h2o_model_{timestamp}.zip"

# The full path will now be within the current 'model' directory
full_mojo_path = os.path.join(output_directory, model_filename)

# Download the MOJO. The 'path' argument is where H2O will save it.
# We want it in the current directory, which is './model'.
mojo_path = model.download_mojo(path=output_directory, get_genmodel_jar=True)

# H2O's download_mojo often saves with a default name (e.g., based on model ID).
# We then rename it to our desired timestamped name.
actual_downloaded_name = mojo_path
desired_new_name = full_mojo_path

if actual_downloaded_name != desired_new_name:
    try:
        os.rename(actual_downloaded_name, desired_new_name)
        print(f"MOJO model renamed from {os.path.basename(actual_downloaded_name)} to: {os.path.basename(desired_new_name)}")
        print(f"MOJO model successfully saved to: {desired_new_name}")
    except OSError as e:
        print(f"Error renaming MOJO file: {e}")
        print(f"MOJO model still available at its original downloaded path: {actual_downloaded_name}")
else:
    print(f"MOJO model saved to: {mojo_path}")

MOJO model renamed from GBM_model_python_1749740346516_2.zip to: my_h2o_model_20250616_134008.zip
MOJO model successfully saved to: ./my_h2o_model_20250616_134008.zip


In [27]:

import h2o
from h2o.estimators import H2OGradientBoostingEstimator

h2o.init()
data = h2o.import_file("https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv")

predictors = data.columns[:-1]
response = data.columns[-1]
data[response] = data[response].asfactor()

model = H2OGradientBoostingEstimator(ntrees=50, max_depth=5, learn_rate=0.1)
model.train(x=predictors, y=response, training_frame=data)

mojo_path = model.download_mojo(path=".", get_genmodel_jar=True)
print(f"MOJO model saved to: {mojo_path}")

Checking whether there is an H2O instance running at http://localhost:54321. connected.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,4 days 0 hours 47 mins
H2O_cluster_timezone:,Asia/Kolkata
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.1
H2O_cluster_version_age:,"2 years, 4 months and 8 days"
H2O_cluster_name:,H2O_from_python_mdshafiuddin_qae39y
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,4.443 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
gbm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
MOJO model saved to: /Users/mdshafiuddin/Desktop/testiat/mlopspoc/model/GBM_model_python_1749740346516_6.zip
